In [1]:
import h5py
import numpy as np
import os
from pathlib import Path
main_dir = str(Path().resolve())

In [2]:
with h5py.File(os.path.join(main_dir,'gaze_capture','gaze_data',f"Bottle_gaze_data_2.h5"),'r') as gaze_data:
    unstr_gaze_dataset = {k:v[()] for k,v in gaze_data.items()}
def get_unstr_batch(size):
    data = unstr_gaze_dataset['0']
    data_ind = np.random.choice(len(data),size)
    return data[data_ind]

In [3]:
target_pos_dataset = np.load(os.path.join(main_dir,'demos','Bottle_model_on_policy_100_debug.npy'),allow_pickle=True)
target_pos_dataset = np.array(sum([[info['target_pos'] for info in traj['env_infos']] for traj in target_pos_dataset],[]))
def get_target_pos(size):
    data = target_pos_dataset
    data_ind = np.random.choice(len(data),size)
    batch = data[data_ind]
    batch = np.concatenate((batch,np.zeros((batch.shape[0],gaze_dim-3))),axis=1)
    return batch

In [4]:
from rlkit.torch.networks import Mlp
from torch import optim
gaze_dim = 128
hidden_sizes = (64,64)
lr=5e-4
encoder = Mlp(input_size=gaze_dim,
               output_size=gaze_dim,
               hidden_sizes=hidden_sizes
               )
encoder_inv = Mlp(hidden_sizes=hidden_sizes,
                    input_size=gaze_dim,
                    output_size=gaze_dim,
                    layer_norm=True)
enc_disc = Mlp(hidden_sizes=hidden_sizes,
                    input_size=gaze_dim,
                    output_size=1,
                    layer_norm=True)
enc_inv_disc = Mlp(hidden_sizes=hidden_sizes,
                    input_size=gaze_dim,
                    output_size=1,
                    layer_norm=True)
optimizer = optim.Adam(
    list(encoder.parameters())+list(encoder_inv.parameters()),
    lr=lr,
)
disc_optimizer = optim.Adam(
    list(enc_disc.parameters())+list(enc_inv_disc.parameters()),
    lr=lr,
)

In [5]:
from tqdm import tqdm
import torch as th
import torch.nn.functional as F
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt

epochs = 100
t = tqdm(range(epochs))
for epoch in t:
    x = th.tensor(get_unstr_batch(128))
    y = th.tensor(get_target_pos(128)).float()
    pred_y = encoder(x)
    pred_x = encoder_inv(y)
    recon_x = encoder_inv(pred_y)
    recon_y = encoder(pred_x)

    # measure l1 loss of reconstruction
    x_recon_loss = F.l1_loss(recon_x,x)
    y_recon_loss = F.l1_loss(recon_y,y)
    loss = x_recon_loss + y_recon_loss
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print_x_loss = np.around(x_recon_loss.item(),3)
    print_y_loss = np.around(y_recon_loss.item(),3)
    t.set_description("gaze recon: "+str(print_x_loss)+" target pos recon: "+str(print_y_loss))
    
fig = plt.figure()
ax = Axes3D(fig)
# ax.scatter(*(pred_y.numpy().T),label="predicted target positions")
# ax.scatter(*(y.numpy().T),label="predicted target positions")
# plt.legend()
# plt.pause(0.05)
# plt.show()
#     plt.close()

ImportError: Failed to import any qt binding